In [1]:
] activate ../.

In [2]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Reel
using Revise

using EmergencyBrakingSystem


In [3]:

# Definition ego vehicle and pedestrian behavior

scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA", "FP"]
vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6
hit_points = [0., 10., 20., 30., 40., 50.]


# FP Tests
#CPAN25 -100, 200

ego_v = vut_speeds[1]
hit_point = hit_points[1]
scenario = scenarios[2]


### Simulate scenario with parameters above defined
# generate scenario based on scenario type
(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = generate_scenario(scenario, ego_v, hit_point)
# simulate scenario
(rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
#evaluate result
(collision2, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision, ped_x)

#println(ego_x, " ", ego_y, " ", ego_v, " ", ped_x, " ", ped_y, " ", ped_v, " ", ped_theta, " ", obstacles)

(false, true, 0.0, 2.612599206349203, -8.125, 20.0, -10.0)

In [4]:
# Visualize scenario
cam = StaticCamera(VecE2(95.0,0.0),20.0)
#cam = CarFollowCamera(1,10.0)

duration, fps, render_hist = EmergencyBrakingSystem.animate_record(rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, emergency_brake_request, prediction_obstacle, cam)
film = roll(render_hist, fps = fps, duration = duration)
#write("file.mp4", film) # Write to a webm video

Frames{MIME{Symbol("image/png")}}("/tmp/tmpWSZZ9x", 0x0000000000000065, 20.0, nothing)

In [5]:
# Evaluate EuroNCAP scenarios

using CSV
using DataFrames


algorithm = "EmergancyBrakingSystem"


policy = "no"


log_filename = string("results_", algorithm, "_", policy, ".csv")

println(log_filename)

scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA", "FP"]
vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6


println("Algorithm to evaluate: ", algorithm, " Policy: ", policy, " (if specified)")

# Evaluate all EuroNCAP scenarios
rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, emergency_brake_request, prediction_obstacle, collision, ego_a
results = Vector[]
for scenario in scenarios
    if (scenario == "FP") 
        hit_points = [-100., 200.]
    else
        hit_points = [0., 10., 20., 30., 40., 50.]
    end
    for hit_point in hit_points
        for vut_speed in vut_speeds
            ego_v = vut_speed
            (ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = EmergencyBrakingSystem.generate_scenario(scenario, ego_v, hit_point)
            if algorithm == "EmergancyBrakingSystem"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
            elseif algorithm == "PedestrianAvoidancePOMDP"

            elseif algorithm == "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"
                
            else
                println("No valid algorithm defined!")
                return false
            end
            
            (collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = EmergencyBrakingSystem.evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision, ped_x)
            println("Collision: ", collision, " eb: ", emergency_brake_intervention, " ", dv_collision, " ", v_mean, " ", a_mean, " ", a_jerk, " ", a_min)
            result = [scenario_id, ego_v, hit_point, collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min  ]
            push!(results, result)
        end
    end 
end  


# store results from scneario evaluation in log file
df = DataFrame(results)
df = DataFrame(Matrix(df)')
rename!(df, :x1 => :scenario_id, :x2 => :ego_v, :x3 => :hit_point, :x4 => :collision, :x5 => :eb_intervention)
rename!(df, :x6 => :dv_collision, :x7 => :v_mean, :x8 => :a_mean, :x9 => :a_jerk, :x10 => :a_min)

CSV.write(log_filename, df);

 

(sum_collisions, sum_eb, dv, v_mean, a_mean, a_jerk, a_min) = EmergencyBrakingSystem.evaluateScenariosMetric(results)

println("#collisions: ", sum_collisions, " #emergency brake interventions: ", sum_eb, " dv_m: ", dv, " v_mean: ", v_mean, " a_mean: ", a_mean, " a_jerk: ", a_jerk, " a_min: ", a_min)

df


results_EmergancyBrakingSystem_no.csv
Algorithm to evaluate: EmergancyBrakingSystem Policy: no (if specified)
false true 0.0 2.612599206349203 -8.125 20.0 -10.0
false true 0.0 3.8912429378531037 -8.5 20.0 -10.0
false true 0.0 5.219489981785057 -8.846153846153847 20.0 -10.0
false true 0.0 6.186752136752145 -9.0625 20.0 -10.0
false true 0.0 7.2512437810945185 -9.210526315789474 20.0 -10.0
true true 5.472222222222221 9.513888888888888 -8.636363636363637 10.0 -10.0
true true 3.8611111111111107 10.436507936507923 -9.117647058823529 10.0 -10.0
true true 5.75 11.915322580645162 -9.0625 10.0 -10.0
true true 6.638888888888889 13.107142857142874 -9.117647058823529 10.0 -10.0
true true 11.527777777777777 15.069444444444464 -8.5 10.0 -10.0
true true 12.916666666666668 16.458333333333318 -8.5 10.0 -10.0
false true 0.0 2.6095959595959566 -8.125 20.0 -10.0
false true 0.0 3.8958333333333295 -8.5 20.0 -10.0
false true 0.0 5.219489981785057 -8.846153846153847 20.0 -10.0
false true 0.0 6.186752136752145 

false true 0.0 10.742753623188406 -9.444444444444445 20.0 -10.0
false true 0.0 11.968599033816442 -9.5 20.0 -10.0
false true 0.0 12.071111111111138 -9.545454545454545 20.0 -10.0
false true 0.0 13.033333333333314 -9.571428571428571 20.0 -10.0
false true 0.0 2.612599206349203 -8.125 20.0 -10.0
false true 0.0 3.8958333333333295 -8.5 20.0 -10.0
false true 0.0 5.219489981785057 -8.846153846153847 20.0 -10.0
false true 0.0 6.186752136752145 -9.0625 20.0 -10.0
false true 0.0 7.2512437810945185 -9.210526315789474 20.0 -10.0
false true 0.0 8.391339869281039 -9.285714285714286 20.0 -10.0
false true 0.0 9.378968253968239 -9.375 20.0 -10.0
false true 0.0 10.742753623188406 -9.444444444444445 20.0 -10.0
false true 0.0 11.968599033816442 -9.5 20.0 -10.0
false true 0.0 12.071111111111138 -9.545454545454545 20.0 -10.0
false true 0.0 13.033333333333314 -9.571428571428571 20.0 -10.0
false true 0.0 2.612599206349203 -8.125 20.0 -10.0
false true 0.0 3.8958333333333295 -8.5 20.0 -10.0
false true 0.0 5.2194

#collisions: 13 #emergency brake interventions: 264 dv_m: 0.3194444444444445 v_mean: 8.425099261433875 a_mean: -8.409117576530365 a_jerk: 18.006993006993007 a_min: -9.23076923076923


,scenario_id,ego_v,hit_point,collision,eb_intervention,dv_collision,v_mean,a_mean,a_jerk,a_min
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,2.77778,0.0,0.0,1.0,0.0,2.6126,-8.125,20.0,-10.0
2,1.0,4.16667,0.0,0.0,1.0,0.0,3.89124,-8.5,20.0,-10.0
3,1.0,5.55556,0.0,0.0,1.0,0.0,5.21949,-8.84615,20.0,-10.0
4,1.0,6.94444,0.0,0.0,1.0,0.0,6.18675,-9.0625,20.0,-10.0
5,1.0,8.33333,0.0,0.0,1.0,0.0,7.25124,-9.21053,20.0,-10.0
6,1.0,9.72222,0.0,1.0,1.0,5.47222,9.51389,-8.63636,10.0,-10.0
7,1.0,11.1111,0.0,1.0,1.0,3.86111,10.4365,-9.11765,10.0,-10.0
8,1.0,12.5,0.0,1.0,1.0,5.75,11.9153,-9.0625,10.0,-10.0
9,1.0,13.8889,0.0,1.0,1.0,6.63889,13.1071,-9.11765,10.0,-10.0
